In [1]:
import numpy as np

In [2]:
import sys
sys.path.append("..")
import pyadlml

In [3]:
import numpy as np
from pandas import DataFrame
from pandas._libs.index import timedelta
import pandas as pd

from pyadlml.dataset.util import fill_nans_ny_inverting_first_occurence
from pyadlml.dataset._dataset import Data, correct_activity_overlap, \
    _dev_rep1_to_rep2, correct_device_ts_duplicates, \
    _is_activity_overlapping, correct_device_rep3_ts_duplicates, \
    device_rep2_2_rep3, device_rep3_2_rep2, \
    ACTIVITY, VAL, START_TIME, END_TIME, TIME, NAME, DEVICE

def _load_devices(device_fp):
    sens_labels = {
        1: 'Microwave', 
        5: 'Hall-Toilet door',
        6: 'Hall-Bathroom door',
        7: 'Cups cupboard',
        8: 'Fridge',
        9: 'Plates cupboard',
        12: 'Frontdoor',
        13: 'Dishwasher',
        14: 'ToiletFlush',
        17: 'Freezer',
        18: 'Pans Cupboard',
        20: 'Washingmachine',
        23: 'Groceries Cupboard',
        24: 'Hall-Bedroom door'
    }
    ide = 'id'
    sens_data = pd.read_csv(device_fp,
                sep="\t",
                skiprows=23,
                skipfooter=1,
                parse_dates=True,
                names=[START_TIME, END_TIME, ide, VAL],
                engine='python' #to ignore warning for fallback to python engine because skipfooter
                #dtype=[]
                )


    #k todo declare at initialization of dataframe
    sens_data[START_TIME] = pd.to_datetime(sens_data[START_TIME])
    sens_data[END_TIME] = pd.to_datetime(sens_data[END_TIME])
    sens_data[VAL] = sens_data[VAL].astype('bool')

    # replace numbers with the labels
    sens_data[DEVICE] = sens_data[ide].map(sens_labels)
    sens_data = sens_data.drop(ide, axis=1)
    sens_data = sens_data.sort_values(START_TIME)
    return sens_data

def load(device_fp, activity_fp):
    df_activities = _load_activities(activity_fp)

    # correct overlapping activities as going to toilet is done in parallel
    # for this dataset >:/
    while _is_activity_overlapping(df_activities):
        df_activities = correct_activity_overlap(df_activities)

    df_dev = _load_devices(device_fp)
    df_dev_rep1 = _dev_rep1_to_rep2(df_dev)
    df_dev_rep2 = df_dev.copy()
    
    # correct possible duplicates for the devices
    df_dev_rep1 = correct_device_ts_duplicates(df_dev_rep1)

    # correct possible duplicates for representation 2
    rep3 = device_rep2_2_rep3(df_dev_rep2)
    cor_rep3 = correct_device_rep3_ts_duplicates(rep3)
    df_dev_rep2 = device_rep3_2_rep2(cor_rep3)

    data = Data(df_activities, df_dev_rep1)
    data.df_dev_rep2 = df_dev_rep2
    data.df_dev_rep3 = cor_rep3
    return data



ImportError: cannot import name '_dev_rep1_to_rep2' from 'pyadlml.dataset._dataset' (../pyadlml/dataset/_dataset.py)

In [4]:
data_path = "/home/chris/code/adlml/datasets/casas_aruba/corrected_data.csv"

#data = kasteren.load(sensorData, activityData)


In [5]:
df = pd.read_csv(data_path,
                sep=",",
                #parse_dates=True,
                infer_datetime_format=True,
                na_values=None,
                names=[START_TIME, 'id', VAL, ACTIVITY],
                engine='python' #to ignore warning for fallback to python engine because skipfooter
                #dtyp
                )
df[START_TIME] = pd.to_datetime(df[START_TIME])

In [8]:
df_act = _get_activity_df(df)



In [9]:
def device_rep3_2_rep2(df_rep3):
    """
    transforms a device representation 3 into 2
    params: df: pd.DataFrame
                rep3: col (time, device, val)
                example row: [2008-02-25 00:20:14, Freezer, False]
    returns: df: (pd.DataFrame)
                rep2: columns are (start time, end_time, device)
                example row: [2008-02-25 00:20:14, 2008-02-25 00:22:14, Freezer]         
    """
    df = df_rep3.copy().reset_index()
    df = df.sort_values('time')
    df['ones'] = 1

    df_start = df[df['val']]
    df_end = df[~df['val']]

    df_end.rename(columns={'time': 'end_time'}, inplace=True)
    df_start.rename(columns={'time': 'start_time'}, inplace=True)
   
    df_end['pairs'] = df_end.groupby(['device'])['ones'].apply(lambda x: x.cumsum())
    df_start['pairs'] = df_start.groupby(['device'])['ones'].apply(lambda x: x.cumsum())        
    
    
    df = pd.merge(df_start, df_end, on=['pairs', 'device'])
    df['val'] = True
    df = df.sort_values('start_time')
    
    #assert int(len(df_rep3)/2) == len(df), 'Somewhere two following events of the'    
    return df[['start_time', 'end_time', 'val', 'device']]
device_rep3_2_rep2(rep3)

NameError: name 'rep3' is not defined

In [ ]:
rep2 = 

In [14]:
def correct_device_rep3_ts_duplicates(df):
    """
    """
    
    dup_mask = df.duplicated(subset=[TIME], keep=False)
    duplicates = df[dup_mask]
    print(duplicates)
    uniques = df[~dup_mask]

    # for every pair of duplicates add a millisecond on the second one
    duplicates = duplicates.reset_index(drop=False)
    s = duplicates['time'] + pd.Timedelta(milliseconds=1)
    mask = (duplicates.index % 2 == 0) # every second index
    duplicates['time'] = duplicates['time'].where(mask, s)
    duplicates = duplicates.set_index('index')

    # concatenate and sort the dataframe 
    df = pd.concat([duplicates, uniques], sort=True)
    
    # set the time as index again
    df = df.sort_values(TIME)
    print(df)
    assert df[TIME].is_unique, 'Time is not unique'

    df = df.set_index(TIME)
    return df

#print(rep3.head())
correct_device_rep3_ts_duplicates(rep3)

                              time device    val
1564867 2011-05-23 00:47:17.505530   M003   True
1564868 2011-05-23 00:47:22.304470   M003  False
1564869 2011-05-23 00:47:57.755188   M003   True
1564870 2011-05-23 00:48:00.224731   M003  False
1564878 2011-05-23 01:51:36.426614   M003   True
...                            ...    ...    ...
1577697 2011-05-23 21:45:58.597750   M003   True
1577698 2011-05-23 21:46:04.542391   M002  False
1577699 2011-05-23 21:46:13.713594   M003  False
1577700 2011-05-23 21:46:14.786178   M003   True
1577701 2011-05-23 21:46:30.496566   M003  False

[12516 rows x 3 columns]
        device                       time    val
0         M003 2010-11-04 00:03:50.209589   True
1         M003 2010-11-04 00:03:57.399391  False
15        M003 2010-11-04 02:32:33.351906   True
16        M003 2010-11-04 02:32:38.895958  False
20        M003 2010-11-04 03:42:21.823650   True
...        ...                        ...    ...
1719544   M002 2011-06-11 22:33:01.209214  

AssertionError: Time is not unique

In [10]:
df_dev = _get_devices_df(df)
rep3 = df_dev.copy()
cor_rep3 = correct_device_rep3_ts_duplicates(rep3)
#rep1 = _dev_rep1_to_rep2(df_dev)
#rep3 = device_rep2_2_rep3(rep2)


AssertionError: Time is not unique

In [ ]:
#rep1
df

In [ ]:
rep2

In [ ]:
rep3

In [ ]:
def load(device_fp, activity_fp):
    df_activities = _load_activities(activity_fp)

    # correct overlapping activities as going to toilet is done in parallel
    # for this dataset >:/
    while _is_activity_overlapping(df_activities):
        df_activities = correct_activity_overlap(df_activities)

    df_dev = _load_devices(device_fp)
    df_dev_rep1 = _dev_rep1_to_rep2(df_dev)
    df_dev_rep2 = df_dev.copy()
    
    # correct possible duplicates for the devices
    df_dev_rep1 = correct_device_ts_duplicates(df_dev_rep1)

    # correct possible duplicates for representation 2
    rep3 = device_rep2_2_rep3(df_dev_rep2)
    cor_rep3 = correct_device_rep3_ts_duplicates(rep3)
    df_dev_rep2 = device_rep3_2_rep2(cor_rep3)

    data = Data(df_activities, df_dev_rep1)
    data.df_dev_rep2 = df_dev_rep2
    data.df_dev_rep3 = cor_rep3
    return data
